In [1]:
import textacy
import spacy
spacy.load('en')
import pandas as pd
import numpy as np

Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
okc = pd.read_csv('../Assets/A/profiles.csv')

In [3]:
okc.shape

(59946, 31)

In [4]:
essay = pd.read_csv('../Assets/A/one_long_essay.csv')

In [5]:
essay.shape

(57809, 23)

In [6]:
essay.essays.iloc[:50,]

0     about me i would love to think that i was some...
1     i am a chef this is what that means i am a wor...
2     i m not ashamed of much but writing public tex...
3     i work in a library and go to school reading t...
4     hey how s it going currently vague on the prof...
5     i m an australian living in san francisco but ...
6     life is about the little things i love to laug...
7     writing meeting new people spending time with ...
8     oh goodness at the moment i have jobs so it d ...
9     my names jake i m a creative guy and i look fo...
10    update i m seeing someone so off the market i ...
11    i was born in wisconsin grew up in iowa and mo...
12                                    bang my shit bang
13    i have an awesome career working as a senior m...
14    dancing playing exploring smiling and doing my...
15    i just moved to the bay area from austin tx or...
16    to sum myself in whole i have adventurous tend...
17    some of my favorite things riding my motor

In [7]:
essay = essay.drop('Unnamed: 0', axis=1)

# rerun feature processing on train.csv!!!! ASAP!

In [8]:
essay.head()

,age,body_type,diet,drinks,drugs,education,ethnicity,height,income,job,...,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status,essays
0,22,a little extra,strictly anything,socially,never,working on college/university,"asian, white",75.0,-1,transportation,...,"doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single,about me i would love to think that i was some...
1,35,average,mostly other,often,sometimes,working on space camp,white,70.0,80000,hospitality / travel,...,"doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (...",single,i am a chef this is what that means i am a wor...
2,38,thin,anything,socially,NaN,graduated from masters program,NaN,68.0,-1,NaN,...,NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available,i m not ashamed of much but writing public tex...
3,23,thin,vegetarian,socially,NaN,working on college/university,white,71.0,20000,student,...,doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,"english, german (poorly)",single,i work in a library and go to school reading t...
4,29,athletic,NaN,socially,never,graduated from college/university,"asian, black, other",66.0,-1,artistic / musical / writer,...,NaN,straight,likes dogs and likes cats,NaN,m,aquarius,no,english,single,hey how s it going currently vague on the prof...


In [9]:
essay.columns

Index([u'age', u'body_type', u'diet', u'drinks', u'drugs', u'education',
       u'ethnicity', u'height', u'income', u'job', u'last_online', u'location',
       u'offspring', u'orientation', u'pets', u'religion', u'sex', u'sign',
       u'smokes', u'speaks', u'status', u'essays'],
      dtype='object')

In [12]:
# Change essays column to unicode

def decode(essay):
    return essay.decode('utf-8')

essay['essays'] = essay.essays.apply(decode)

In [13]:
essay_d = essay.to_dict(orient='records')

In [14]:
type(essay_d)

list

In [18]:
# Save text as content_stream, holding onto other info as metadata
content_stream, metadata_stream = textacy.fileio.split_content_and_metadata(essay_d, 'essays', itemwise=False)

In [19]:
corpus = textacy.TextCorpus.from_texts('en', content_stream, metadata_stream, n_threads=2)

In [20]:
doc_term_matrix, id2term = corpus.as_doc_term_matrix(
    (doc.as_terms_list(words=True, ngrams=False, named_entities=True)
     for doc in corpus),
    weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.95)

In [21]:
model = textacy.tm.TopicModel('nmf', n_topics=10)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)

### Run Textacy Topic Model on Full Essay Texts

In [22]:
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=10):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'm   t   like   thing   think   people   ve   time   good   know')
('topic', 1, ':', u'love   life   enjoy   friend   laugh   travel   great   smile   good   live')
('topic', 2, ':', u'be   not   lol   u   know   n   can   good   ill   money')
('topic', 3, ':', u'art   music   artist   design   creative   film   draw   photography   painting   gallery')
('topic', 4, ':', u'facebook   com   www   inspiring   design   profile   delete   piercings   reddit   kinky')
('topic', 5, ':', u'fun   look   like   have   meet   work   want   guy   friend   go')
('topic', 6, ':', u'year   work   san   francisco   travel   city   live   bay   sf   friend')
('topic', 7, ':', u'movie   friend   food   family   music   love   favorite   like   hang   watch')
('topic', 8, ':', u'new   try   thing   meet   people   want   restaurant   explore   friend   interesting')
('topic', 9, ':', u'youtube   com   http   www   v   watch   video   feature   av   ob')


In [23]:
model = textacy.tm.TopicModel('nmf', n_topics=15)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=10):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u't   thing   m   like   people   think   time   s   good   know')
('topic', 1, ':', u'be   not   lol   u   know   good   n   guy   go   money')
('topic', 2, ':', u'm   ve   ll   pretty   t   look   guy   d   go   sure')
('topic', 3, ':', u'love   laugh   music   favorite   movie   food   smile   cook   life   family')
('topic', 4, ':', u'schrute   sicily   auburn   spaniard   unsettling   license   admin   about month   tmnt   kingpin')
('topic', 5, ':', u'art   music   artist   design   creative   photography   draw   film   painting   gallery')
('topic', 6, ':', u'dance   salsa   music   dancing   hip   hop   swing   tango   party   food')
('topic', 7, ':', u'friend   new   work   family   travel   try   enjoy   go   fun   food')
('topic', 8, ':', u'youtube   com   http   www   v   watch   feature   video   facebook   av')
('topic', 9, ':', u'like   fun   movie   good   food   kind   music   thing   watch   go')
('topic', 10, ':', u'game   video   play   movie   th

In [26]:
model = textacy.tm.TopicModel('nmf', n_topics=3)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=10):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'm   like   love   t   friend   good   thing   time   people   work')
('topic', 1, ':', u'be   not   lol   u   love   like   fun   good   work   go')
('topic', 2, ':', u'youtube   com   http   www   v   watch   video   feature   facebook   av')


In [102]:
## REFINE THIS FUNCTION
def find_topics(condition1, condition2, n_topics=10, top_n=10, max_df=.95):
    filtered_corpus = list(corpus.get_docs(
            lambda doc: doc.metadata[condition1] == condition2))
    # Build dtm from filtered corpus
    doc_term_matrix, id2term = corpus.as_doc_term_matrix(
    (doc.as_terms_list(words=True, ngrams=False, named_entities=True)
     for doc in filtered_corpus),
    weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=max_df)
    # Fit topic model
    model = textacy.tm.TopicModel('nmf', n_topics=n_topics)
    model.fit(doc_term_matrix)
    doc_topic_matrix = model.transform(doc_term_matrix)
    for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=top_n):
        print('topic', topic_idx, ':', '   '.join(top_terms))
    

### Run topic model just on essays by men

In [103]:
find_topics('sex','m')

('topic', 0, ':', u'm   like   t   thing   good   time   people   think   love   ve')
('topic', 1, ':', u'be   not   u   like   ill   n   good   know   alot   can')
('topic', 2, ':', u'youtube   com   www   http   v   watch   facebook   feature   ob   av')
('topic', 3, ':', u'new   meet   try   explore   bay   city   thing   people   san   area')
('topic', 4, ':', u'friend   love   family   work   food   movie   music   good   sport   life')
('topic', 5, ':', u'money   goin   make   drum   cool   nd   car   retire   want   current')
('topic', 6, ':', u'woman   sex   want   relationship   man   treat   love   mature   honest   share')
('topic', 7, ':', u'fun   have   like   meet   want   sex   mind   open   funny   work')
('topic', 8, ':', u'look   enjoy   guy   relationship   live   real   smile   girl   good   share')
('topic', 9, ':', u'lol   soccer   haha   not   guy   know   nice   lazy   phone   ask')


### ... And just by women

In [104]:
find_topics('sex','f')

('topic', 0, ':', u'like   friend   good   t   love   thing   time   people   work   music')
('topic', 1, ':', u'm   t   ve   pretty   ll   d   think   game   right   lot')
('topic', 2, ':', u'be   not   school   can   hey   son   go   look   wanna   know')
('topic', 3, ':', u'u   n   wana   daughter   tell   kno   jus   chihuahua   beverly   know')
('topic', 4, ':', u'shiny   bright   go   sheep   laze   one year   warning   conversate   pass   display')
('topic', 5, ':', u'lol   phone   future   time   bourne   favorite   relax   starbucks   know   movie')
('topic', 6, ':', u'dance   salsa   music   dancing   tango   hip   hop   swing   latin   study')
('topic', 7, ':', u'brasil   requirement   laze   one year   warning   conversate   pass   display   flag   african')
('topic', 8, ':', u'love   life   smile   family   friend   travel   full   live   fun   enjoy')
('topic', 9, ':', u'brasil   requirement   laze   one year   warning   conversate   pass   display   flag   african')


Why the repeated topic?

In [105]:
find_topics('sex','f', 20)

('topic', 0, ':', u'like   thing   love   people   m   good   music   food   movie   time')
('topic', 1, ':', u'm   ve   pretty   ll   game   d   right   t   time   try')
('topic', 2, ':', u'love   life   smile   music   live   eye   heart   cook   family   dance')
('topic', 3, ':', u'be   u   school   hey   son   n   not   look   wanna   go')
('topic', 4, ':', u'brasil   requirement   laze   one year   warning   conversate   pass   display   flag   african')
('topic', 5, ':', u'brasil   requirement   laze   one year   warning   conversate   pass   display   flag   african')
('topic', 6, ':', u't   like   haven   know   profile   think   message   win   isn   time')
('topic', 7, ':', u'brasil   requirement   laze   one year   warning   conversate   pass   display   flag   african')
('topic', 8, ':', u'art   music   creative   artist   photography   gallery   create   paint   adventure   theater')
('topic', 9, ':', u'brasil   requirement   laze   one year   warning   conversate   pass  

why does this repeat the same phrase so many times?
u'brasil   requirement   laze   one year   warning   conversate   pass   display   flag   african   mindless   moderately   sheep   pow   forbid'

Is there a repeated essay in essays?

In [66]:
def find_bad_string(essay):
    if essay.find('conversate') >= 0:
        print essay

In [ ]:
essay.essays.apply(find_bad_string)

### How about people who drink a lot?

In [61]:
drunk_essays = list(corpus.get_docs(
            lambda doc: doc.metadata['drinks'] == 'desperately' or doc.metadata['drinks'] == 'very often'))

In [62]:
len(drunk_essays)

66

In [65]:
doc_term_matrix, id2term = corpus.as_doc_term_matrix(
    (doc.as_terms_list(words=True, ngrams=False, named_entities=True)
     for doc in drunk_essays),
    weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.95)
model = textacy.tm.TopicModel('nmf', n_topics=10)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=15):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'm   like   t   think   good   time   thing   music   love   work   life   people   food   movie   book')
('topic', 1, ':', u'youtube   com   http   www   v   watch   facebook   netflix   job   capable   beatles   state   improve   c   store')
('topic', 2, ':', u'n   wanna   bring   city   bike   party   guy   friend   livin   gal   brew   whiskey   wild   dude   da')
('topic', 3, ':', u'yahoo   notebook   com   play   coffee   family   want   love   friend   movie   music   simply   table   point   jack')
('topic', 4, ':', u'hair   salon   intern   pepper   dr   blow   rap   love   different   animal   interested   dance   girl   school   fun')
('topic', 5, ':', u'cheeseburger   double   t   gotta   pickle   color   tomato   chick   fat   burger   king   beef   clean   cheese   movie')
('topic', 6, ':', u'amateur   advice   answer   avoid   high   musical   musician   jam   reason   be   sport   conversation   message   hard   grow')
('topic', 7, ':', u'foreign   wi

### And people who don't drink

In [106]:
find_topics('drinks','not at all')

('topic', 0, ':', u'm   like   love   t   good   friend   thing   people   time   music')
('topic', 1, ':', u'm   family   love   n   lil   minded   latin   hi   earth   new')
('topic', 2, ':', u'be   not   box   work   random   school   bite   smoke   friendly   u')
('topic', 3, ':', u'timey   abuse   kuti   karma   african   american   intimacy   software   earth   happiness')
('topic', 4, ':', u'not   gym   soda   be   thing   can   candy   gear   dumb   bbq')
('topic', 5, ':', u'marine   corp   ll   angeles   recruit   edible   ship   patrol   drill   seether')
('topic', 6, ':', u'meditate   bread   meditation   swim   mission   classic   house   think   work   music')
('topic', 7, ':', u'timey   abuse   kuti   karma   african   american   intimacy   software   earth   happiness')
('topic', 8, ':', u'tom   tall   sixth   bass   band   stranger   die   meat   peeve   snl')
('topic', 9, ':', u'jesus   bible   youtube   record   guitar   aa   church   bfa   narnia   dodgeball')


### Explore some job categories

In [79]:
essay.job.value_counts()

other                                742
science / tech / engineering         520
student                              513
artistic / musical / writer          469
computer / hardware / software       449
sales / marketing / biz dev          438
medicine / health                    419
education / academia                 367
executive / management               244
entertainment / media                224
banking / financial / real estate    214
law / legal services                 123
hospitality / travel                 114
construction / craftsmanship         108
clerical / administrative             66
political / government                63
rather not say                        36
transportation                        35
unemployed                            25
retired                               25
military                              17
Name: job, dtype: int64

In [107]:
find_topics('job', 'science / tech / engineering')

('topic', 0, ':', u'm   like   love   t   friend   good   thing   work   time   new')
('topic', 1, ':', u'spontaneous   concert   outdoors   adventure   run   hang   travel   family   friend   facto')
('topic', 2, ':', u'youtube   http   v   www   com   watch   skinny   ha   chemistry   let')
('topic', 3, ':', u'be   not   gonna   work   project   mind   fun   thing   friend   want')
('topic', 4, ':', u'lobos   suck   mug   cuddle   literally   commercial   apartment   daughter   chivalry   permit')
('topic', 5, ':', u'lobos   suck   mug   cuddle   literally   commercial   apartment   daughter   chivalry   permit')
('topic', 6, ':', u'lobos   suck   mug   cuddle   literally   commercial   apartment   daughter   chivalry   permit')
('topic', 7, ':', u'lobos   suck   mug   cuddle   literally   commercial   apartment   daughter   chivalry   permit')
('topic', 8, ':', u'enjoy   look   live   lobos   commercial   apartment   daughter   chivalry   permit   merely')
('topic', 9, ':', u'lobos 

In [108]:
find_topics('job', 'student')

('topic', 0, ':', u'm   like   love   t   people   time   friend   good   thing   music')
('topic', 1, ':', u'be   not   u   good   know   wrestle   eyelash   highschool   friend   n')
('topic', 2, ':', u'summer   interning   fran   graphic   year   roommate   design   degree   graduate   san')
('topic', 3, ':', u'mac   cellphone   lunch   specific   discussion   hang   number   friend   wanna   reading')
('topic', 4, ':', u'art   concept   game   school   draw   film   artist   gallery   video   painting')
('topic', 5, ':', u'want   hang   talk   fun   new   friend   look   love   meet   marine')
('topic', 6, ':', u'basketball   piano   school   go   fucking   weakerthans   rocket   bright   prince   previous')
('topic', 7, ':', u'meet   student   one   favorite   new   people   thing   previous   pope   fit')
('topic', 8, ':', u'flake   professor   concept   rocket   bright   prince   previous   pope   fit   scooter')
('topic', 9, ':', u'lol   goin   haha   money   phone   sex   car 

In [109]:
find_topics('job', 'artistic / musical / writer')

('topic', 0, ':', u'm   like   love   t   music   good   thing   people   friend   time')
('topic', 1, ':', u'art   show   live   music   artist   illustration   owl   blanket   leon   weather')
('topic', 2, ':', u'try   new   fun   design   sketch   grab   chillin   friend   owl   culture')
('topic', 3, ':', u'bf   clumsy   muni   blanket   leon   weather   security   culture   bjrk   k')
('topic', 4, ':', u'bf   clumsy   muni   blanket   leon   weather   security   culture   bjrk   k')
('topic', 5, ':', u'facebook   design   www   com   hop   time   menswear   gladiator   gatsby   dedicated')
('topic', 6, ':', u'bf   clumsy   muni   blanket   leon   weather   security   culture   bjrk   k')
('topic', 7, ':', u'coffee   month   outside   think   louisiana   miami   airstream   ft   mutual   tight')
('topic', 8, ':', u'bf   clumsy   muni   blanket   leon   weather   security   culture   bjrk   k')
('topic', 9, ':', u'ballet   nose   diaper   beginner   natalie   convince   filthy   imm

In [110]:
find_topics('job', 'computer / hardware / software')

('topic', 0, ':', u'm   like   t   love   thing   work   good   friend   people   time')
('topic', 1, ':', u'sex   fun   russian   pant   bedroom   balance   rest   step   woman   eye')
('topic', 2, ':', u'ray   man   exciting   good   fun   producer   murakami   borges   la   young')
('topic', 3, ':', u'producer   charles   ros   hmmm   gamut   borges   la   young   th   cap')
('topic', 4, ':', u'vacation   gotta   text   friendship   later   guitar   wine   hour   see   happy')
('topic', 5, ':', u'producer   charles   ros   hmmm   gamut   borges   la   young   th   cap')
('topic', 6, ':', u'dollar   morning   three   ancestor   flute   brace   dil   tofu   basil   two')
('topic', 7, ':', u'nightclub   salsa   francisco   spend   san   restaurant   one year   bar   new   path')
('topic', 8, ':', u'eccentric   hat   key   wear   funny   software   person   make   music   hesse')
('topic', 9, ':', u'season   lose   rico   puerto   lost   girlfriend   dork   universe   follow   answer')


In [111]:
find_topics('job', 'medicine / health')

('topic', 0, ':', u'm   love   like   friend   work   good   t   thing   time   people')
('topic', 1, ':', u'love   life   spoil   dance   kid   raise   boob   gonna   sharp   preferably')
('topic', 2, ':', u'personal   trainer   free   silent   crossfit   tame   hell   motivate   usual   suppose')
('topic', 3, ':', u'm   like   lb   sexy   asian   nurse   guy   read   travel   insert')
('topic', 4, ':', u'retreat   stunning   equal   gonna   sharp   preferably   reality   hot   freshly   digital')
('topic', 5, ':', u'badminton   skate   physician   assistant   outgoing   lol   short   story   funny   long')
('topic', 6, ':', u'lip   gloss   r   hip   type   b   reggaeton   lotion   m   mmm')
('topic', 7, ':', u'be   not   medic   free   right   alot   school   business   n   med')
('topic', 8, ':', u'looking   companionship   mystery   professional   medical   maybe   old   right   person   year')
('topic', 9, ':', u'retreat   stunning   equal   gonna   sharp   preferably   reality   

### Explore sexual orientation

In [115]:
essay.orientation.value_counts()

straight    5171
gay          543
bisexual     286
Name: orientation, dtype: int64

In [116]:
find_topics('orientation', 'straight')

('topic', 0, ':', u'm   t   like   thing   think   people   good   time   ve   know')
('topic', 1, ':', u'be   not   know   good   can   money   ill   alot   want   school')
('topic', 2, ':', u'u   n   sf   wana   kno   jus   eye   idk   like   school')
('topic', 3, ':', u'youtube   com   www   http   v   watch   feature   video   av   ob')
('topic', 4, ':', u'art   music   artist   creative   design   film   draw   painting   live   photography')
('topic', 5, ':', u'mac   fleetwood   cheese   dre   da   vinci   somthing   style   code   wine')
('topic', 6, ':', u'new   enjoy   friend   life   travel   fun   family   live   look   work')
('topic', 7, ':', u'lol   soccer   not   guy   haha   know   phone   nice   laugh   people')
('topic', 8, ':', u'friend   movie   like   food   family   music   hang   sport   work   favorite')
('topic', 9, ':', u'love   life   dance   laugh   music   smile   favorite   family   eye   movie')


In [117]:
find_topics('orientation', 'gay')

('topic', 0, ':', u'm   like   love   t   friend   good   people   time   work   life')
('topic', 1, ':', u'be   not   chill   lol   love   u   ur   go   person   music')
('topic', 2, ':', u'not   can   like   wo   hang   vegetarian   know   precious   baffle   nephew')
('topic', 3, ':', u'arizona   dagger   barrel   shot   nephew   fear   impact   knowledge   pumpkin   alicia')
('topic', 4, ':', u'arizona   dagger   barrel   shot   nephew   fear   impact   knowledge   pumpkin   alicia')
('topic', 5, ':', u'francisco   san   maybe   look   new   m   friend   handsome   singer   native')
('topic', 6, ':', u'arizona   dagger   barrel   shot   nephew   fear   impact   knowledge   pumpkin   alicia')
('topic', 7, ':', u'arizona   dagger   barrel   shot   nephew   fear   impact   knowledge   pumpkin   alicia')
('topic', 8, ':', u'ask   ll   question   blah   pale   nephew   fear   impact   knowledge   pumpkin')
('topic', 9, ':', u'linux   rollerblade   addictive   babylon   quietness   yay  

In [118]:
find_topics('orientation', 'bisexual')

('topic', 0, ':', u'm   love   like   t   thing   people   time   friend   life   want')
('topic', 1, ':', u'm   lb   asian   nurse   sexy   like   guy   travel   read   damned')
('topic', 2, ':', u'damned   development   bjrk   k   burroughs   childhood   dancer   augusten   arcade   calm')
('topic', 3, ':', u'be   n   not   digital   easy   lesson   differ   st   bmx   year')
('topic', 4, ':', u'peanut   tree   give   type   m   listen   music   damned   childhood   dancer')
('topic', 5, ':', u'photography   comedian   bjrk   k   burroughs   childhood   dancer   augusten   arcade   calm')
('topic', 6, ':', u'damned   development   bjrk   k   burroughs   childhood   dancer   augusten   arcade   calm')
('topic', 7, ':', u'tacos   idk   yummy   mouth   attend   r   school   m   damned   k')
('topic', 8, ':', u'nothin   yeah   get   damned   comedian   k   burroughs   childhood   dancer   augusten')
('topic', 9, ':', u'damned   development   bjrk   k   burroughs   childhood   dancer   au

### Explore other lifestyle (smoking, drugs, diet)

#### Smokes requires custom condition

In [122]:
essay.smokes.value_counts()

no                4477
sometimes          328
when drinking      293
yes                183
trying to quit     155
Name: smokes, dtype: int64

In [124]:
smokes_essays = list(corpus.get_docs(
            lambda doc: doc.metadata['smokes'] != 'no'))
# Build dtm
doc_term_matrix, id2term = corpus.as_doc_term_matrix(
    (doc.as_terms_list(words=True, ngrams=False, named_entities=True)
     for doc in smokes_essays),
    weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.95)
# Fit model
model = textacy.tm.TopicModel('nmf', n_topics=10)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=15):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'm   like   love   t   music   good   thing   people   time   life   food   friend   movie   work   think')
('topic', 1, ':', u'be   not   lol   u   n   like   know   good   love   alot   life   go   guy   ill   ma')
('topic', 2, ':', u'new   try   meet   explore   bay   restaurant   enjoy   thing   want   friend   soon   people   area   adventure   interesting')
('topic', 3, ':', u'youtube   com   www   http   v   watch   feature   facebook   ob   video   bike   e   bfa   index   art')
('topic', 4, ':', u'fun   work   friend   have   family   good   hang   play   hard   like   look   go   sport   want   lol')
('topic', 5, ':', u'ask   tell   shall   receive   bubbly   tryna   hustle   ll   personality   know   son   question   gym   everthing   relaxation')
('topic', 6, ':', u'admit   have   look   good   private   willing   proficient   sore   sigur   retire   being   goodbye   building   labrador   antoine')
('topic', 7, ':', u'sore   youtu   disabled   retire   b

In [128]:
essay.drugs.value_counts()

never        3746
sometimes     782
often          30
Name: drugs, dtype: int64

In [129]:
drugs_essays = list(corpus.get_docs(
            lambda doc: doc.metadata['drugs'] == 'sometimes' or doc.metadata['drugs'] == 'often'))
# Build dtm
doc_term_matrix, id2term = corpus.as_doc_term_matrix(
    (doc.as_terms_list(words=True, ngrams=False, named_entities=True)
     for doc in drugs_essays),
    weighting='tfidf', normalize=True, smooth_idf=True, min_df=2, max_df=0.95)
# Fit model
model = textacy.tm.TopicModel('nmf', n_topics=10)
model.fit(doc_term_matrix)
doc_topic_matrix = model.transform(doc_term_matrix)
for topic_idx, top_terms in model.top_topic_terms(id2term, top_n=15):
    print('topic', topic_idx, ':', '   '.join(top_terms))

('topic', 0, ':', u'm   like   love   t   thing   good   music   people   time   friend   work   food   life   movie   new')
('topic', 1, ':', u'be   not   lol   n   love   u   ill   sport   chill   good   go   wanna   family   have   live')
('topic', 2, ':', u'ashton   perspective   soggy   feed   crafty   camper   emotionally   breakfast   ultra   stalker   dirty   bitch   communication   tuck   teacher')
('topic', 3, ':', u'ashton   perspective   soggy   feed   crafty   camper   emotionally   breakfast   ultra   stalker   dirty   bitch   communication   tuck   teacher')
('topic', 4, ':', u'ashton   perspective   soggy   feed   crafty   camper   emotionally   breakfast   ultra   stalker   dirty   bitch   communication   tuck   teacher')
('topic', 5, ':', u'funny   stuff   eccentric   person   research   software   hat   key   wear   guy   kind   fun   make   look   music')
('topic', 6, ':', u'youtube   www   v   http   com   watch   feature   e   wikipedia   hour   oakland   skyline 

In [ ]:
### Improve  function to deal with multiple conditions (e.g. "male" and "gay", drinks "very often" or "desperately")
### Try topic model on shorter essays individually?